# 評価指標(分類)

````
・混同行列（Confusion matrix）
・正解率(Accuracy)
・適合率（Precision）
・再現率（Recall）
・F値（F-measure）
````

In [1]:
y_test = [0,0,0,0,0,1,1,1,1,1] # 0をNegative 1をPositiveとする
y_pred = [0,1,0,0,0,0,0,1,1,1]

In [3]:
from sklearn.metrics import confusion_matrix

cmatrix = confusion_matrix(y_test, y_pred)
print(cmatrix)

[[4 1]
 [2 3]]


In [4]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel() #　混同行列のそれぞれの結果を取得
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)

TN 4
FP 1
FN 2
TP 3


## 正解率(Accuracy)

In [5]:
from sklearn.metrics import accuracy_score
print('Accuracy:',accuracy_score(y_test,y_pred))

Accuracy: 0.7


In [6]:
tp, fn, fp, tn = confusion_matrix(y_test, y_pred).ravel() 
print((tp + tn) / (tp + fp + fn + tn)) # 0.7

0.7


## 適合率（Precision）


適合率とは、正事例と予測したもののなかで真の値が正事例の割合を表す指標です。
（出力した結果がどの程度正解していたのかを表す指標）
適合率を重視するときはFN(False Negative)が発生することが許容できるケースです。
例えば、利用シーンは、レコメンドや検索結果などです

In [7]:
from sklearn.metrics import precision_score
print('Precision:', precision_score(y_test,y_pred)) # 0.75

Precision: 0.75


## 再現率（Recall）
再現率とは、真の値が正事例のもののなかで正事例と予測した割合を表す指標です。
再現率を重視するときはFP(False Positive)が発生することが許容できるケースです。
利用シーンは、がん診断などです。
再現率は別名、感度やヒット率、真陽性率とも呼びます。

In [8]:
from sklearn.metrics import recall_score
print('Recall:', recall_score(y_test,y_pred)) # 0.6

Recall: 0.6


## F値（F-measure）
適合率と再現率はトレードオフの関係にあるので、2つの指標をまとめた指標としてF値があります。
F値は、適合率と再現率の調和平均によって計算されます。

In [9]:
from sklearn.metrics import f1_score
print('F1 score:', f1_score(y_test,y_pred)) # 0.67 

F1 score: 0.6666666666666665


In [10]:
from sklearn.metrics import classification_report

print("Classification report")
print(classification_report(y_test, y_pred))

Classification report
              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.75      0.60      0.67         5

    accuracy                           0.70        10
   macro avg       0.71      0.70      0.70        10
weighted avg       0.71      0.70      0.70        10



## 多クラス分類

In [11]:
# 警告文を非表示
import warnings
warnings.simplefilter('ignore')

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
model = LogisticRegression()
model = model.fit(X_train, y_train)
pred = model.predict(X_test)

print(precision_score(y_test, pred,average='macro'))
print(recall_score(y_test, pred,average='macro'))
print(f1_score(y_test, pred,average='macro'))

0.9666666666666667
0.9791666666666666
0.9717034521788342


## 評価方法(交差検証)

・交差検証（Cross-validation）：個々のモデルの汎化性能を評価する手法  
・グリッドサーチ（grid search）：機械学習のハイパーパラメータ探索の方法

In [12]:
# scikit-learnのtrain_test_split()を使うことでデータを分けることが可能です。
# 何も設定しない場合、トレーニングデータには75%、テストデータは25%で分割されます。(test_sizeが25%にデフォルトで設定されています。)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
x = iris.data
y = iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y)

print(len(x_train))
print(len(x_test))

112
38


## 交差検証(クロスバリデーション)

In [14]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

def evaluate_cross_validation(clf, x, y, K):
    cv = KFold(len(y), K,shuffle=True,random_state=0)
    scores = cross_val_score(clf,x,y,cv=cv)
    print(scores)
    print ("Mean score: {} (+/-{})".format( np.mean (scores), sem(scores)))

## グリッドサーチ

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
parameters = [
    {'C': [1, 5, 10, 50],
     'gamma': [0.001, 0.0001]}
]

In [17]:
from sklearn import svm
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 5, 10, 50], 'gamma': [0.001, 0.0001]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [18]:
print(clf.best_params_)

{'C': 1, 'gamma': 0.001}


In [19]:
from sklearn import metrics
def measure_performance(x,y,clf, show_accuracy=True,show_classification_report=True, show_confussion_matrix=True):
    y_pred=clf.predict(x)
    if show_accuracy:
        print("Accuracy:{0:.3f}".format(metrics.accuracy_score(y, y_pred)), "\n")

    if show_classification_report:
        print("Classification report")
        print(metrics.classification_report(y, y_pred), "\n")

    if show_confussion_matrix:
        print("Confussion matrix")
        print(metrics.confusion_matrix(y, y_pred),"\n")